# Naivni Bayesov kasifikator za določanje barve karte

V tej datoteki sem napisal naivni Bayesov klasifikator za ugibanje barve karte glede na njeno besedilo (*oracle text*). Pri tem sem sledil podobnim korakom kot na predavanjih, zato nisem razlagal vsakega posebej.

Iz radovednosti sem napisal dva klasifikatorja, pri prvemu sem vzel zgolj korene besed, pri drugemu pa kar vse besede. Izkazalo se je, da ni nobeden izmed njiju izrazito natančnejši.

Klasifikatorja se nahajata v spodnjem bloku te datoteke.

In [1]:
import pandas as pd
from vzorci_in_pomozne_definicije import FILENAME_KARTE_DODATEK_CSV

In [2]:
podatkovna_baza_kart = pd.read_csv(FILENAME_KARTE_DODATEK_CSV)
pojavitve_barve = podatkovna_baza_kart.groupby("barva").size()
verjetnosti_barve = pojavitve_barve / len(podatkovna_baza_kart)

In [3]:
def odstrani_konce(beseda, konci):
    for konec in konci:
        if beseda.endswith(konec):
            return beseda[:-len(konec)]
    return beseda

konci_ki_jih_bomo_odstranili = ["ing", "ed", "en", "s"]
 
def koren_besede(beseda, ne_naredi_korenov = False):
    beseda = ''.join(znak for znak in beseda if znak.isalpha()) #  
    if not beseda:
        return '$'
    if not ne_naredi_korenov:
        beseda = odstrani_konce(beseda, konci_ki_jih_bomo_odstranili)
        beseda = beseda.rstrip('aeiou')
    if not beseda:
        return '@'
    return beseda

def koreni_besed(niz, ne_naredi_korenov = False):
    return pd.Series(sorted({
        koren_besede(beseda, ne_naredi_korenov=ne_naredi_korenov) for beseda in str(niz).replace(')(', ') (').replace('-', ' ').lower().split() if beseda
    }))

In [4]:
razpredelnica_s_koreni_oracle_textov = podatkovna_baza_kart.oracle_text.apply(koreni_besed)
razpredelnica_s_koreni_oracle_textov_brez_korenov = podatkovna_baza_kart.oracle_text.apply(koreni_besed, ne_naredi_korenov = True)

In [5]:
koreni_tekstov = (
    razpredelnica_s_koreni_oracle_textov.unstack(
    ).reset_index(
    )[
        ["level_1", 0]                           
    ].rename(
        {"level_1": "indeks_oracle_teksta", 0: "koren"},
        axis = 1
    ).dropna(
    )
)

koreni_tekstov_brez_korenov = (
    razpredelnica_s_koreni_oracle_textov_brez_korenov.unstack(
    ).reset_index(
    )[
        ["level_1", 0]                           
    ].rename(
        {"level_1": "indeks_oracle_teksta", 0: "koren"},
        axis = 1
    ).dropna(
    )
)

In [6]:
tabela_barv = podatkovna_baza_kart[["barva"]]
tabela_barv.index.name = "indeks_oracle_teksta"

skupna_tabela = pd.merge(koreni_tekstov, tabela_barv, left_on="indeks_oracle_teksta", right_on="indeks_oracle_teksta")
skupna_tabela_brez_korenov = pd.merge(koreni_tekstov_brez_korenov, tabela_barv, left_on="indeks_oracle_teksta", right_on="indeks_oracle_teksta")

In [7]:
k_obtezitve = 1  # Izkazalo se je, da se koeficient obtežitve nevsebovanosti korena 1 neprimerno bolje kot 0.0001
stolpec_verjetnosti_korenov_pri_oracle_tekstu = skupna_tabela.groupby(["barva", "koren"]).size() / pojavitve_barve
verjetnosti_korenov_pri_oracle_tekstu = stolpec_verjetnosti_korenov_pri_oracle_tekstu.unstack().transpose().fillna(k_obtezitve / len(podatkovna_baza_kart))
stolpec_verjetnosti_korenov_pri_oracle_tekstu_brez_korenov = skupna_tabela_brez_korenov.groupby(["barva", "koren"]).size() / pojavitve_barve
verjetnosti_korenov_pri_oracle_tekstu_brez_korenov = stolpec_verjetnosti_korenov_pri_oracle_tekstu_brez_korenov.unstack().transpose().fillna(k_obtezitve / len(podatkovna_baza_kart))

In [8]:
def ugani_barvo(opis, ne_naredi_korenov = False):
    koreni_oracle_teksta = koreni_besed(opis, ne_naredi_korenov=ne_naredi_korenov)
    if ne_naredi_korenov:
        r = verjetnosti_korenov_pri_oracle_tekstu_brez_korenov.reset_index()
    else:
        r = verjetnosti_korenov_pri_oracle_tekstu.reset_index()
    verjetnosti = r[r.koren.isin(koreni_oracle_teksta)].product(numeric_only=True) * verjetnosti_barve
    return verjetnosti.sort_values(ascending=False).head(10)

Naivna Bayesova klasifikatorja sta končana. Če ju hočete preverjati na kartah, ki jih še ni v podatkovni bazi, je primerna recimo [tale stran](https://www.mtgstocks.com/sets/1183-dominaria-remastered).

Ker sklepam, da marsikdo ne pozna MTG barv, je spodaj preprosta legenda (strogo gledano po pravilih *colorless* ni barva, ampak za potrebe razumevanja klasifikatorja je v redu).

| Barva | Oznaka |
| --- | --- |
| White  | w |
| Blue  | u |
| Black  | b |
| Red  | r |
| Green  | g |
| Colorless | c |

Karta ima lahko od 1 do 5 barv, ali pa je brezbarvna (*colorless*). Če je karta recimo [belo-modra](https://gatherer.wizards.com/pages/card/Details.aspx?multiverseid=457295), to označimo z wu. Vrstni red barv je po pravilih [enolično določen](https://magic.wizards.com/en/articles/archive/ask-wizards-june-2004-2004-06-01).

[Karta v spodnjem primeru](https://gatherer.wizards.com/Pages/Card/Details.aspx?name=Disciple+of+the+Ring) je v resnici modra, torej bi moralo pisati ***u***. Zanimivo je preizkušati kakšne karte, ki jih ni v podatkovnih bazi, recimo večina na [tej strani](https://www.mtgstocks.com/sets/1183-dominaria-remastered).



In [9]:
poskusni_tekst_karte = """
Exile an instant or sorcery card from your graveyard: Choose one —
• Counter target noncreature spell unless its controller pays .
• Disciple of the Ring gets +1/+1 until end of turn.
• Tap target creature.
• Untap target creature.
"""
print("Naivni Bayesov klasifikator, če ustvarimo korene:")
print(ugani_barvo(poskusni_tekst_karte), "\n")
print("Naivni Bayesov klasifikator, če ne ustvarimo korenov:")
print(ugani_barvo(poskusni_tekst_karte, ne_naredi_korenov=True))

Naivni Bayesov klasifikator, če ustvarimo korene:
barva
ur     8.124711e-28
u      4.520522e-30
ubr    1.139376e-30
r      6.238120e-31
wu     9.573188e-32
ub     9.075919e-32
rg     8.641918e-32
wub    4.693130e-32
b      1.730661e-33
w      1.282408e-33
dtype: float64 

Naivni Bayesov klasifikator, če ne ustvarimo korenov:
barva
u      1.968793e-33
ur     1.367819e-33
ub     2.183638e-34
wu     1.080663e-36
ubr    7.873930e-37
w      7.311003e-37
gu     1.449383e-38
b      1.016482e-38
rg     7.057514e-39
r      7.047322e-39
dtype: float64
